<a href="https://colab.research.google.com/github/drdww/OPIM5641/blob/main/Module5/M5_1/2_Assignment_Engineers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Network Problems: Assignment (Engineers)

**OPIM 5641: Business Decision Modeling - University of Connecticut**

--------------------------------------

The inequalities in the textbook could probably be written as equalities (==), but it still works. Great example that generalizes/motivates many other types of machine scheduling problems.

* Please refer to Powell Chapter 10 for more details and examples.
* Pyomo Cookbook (related example): https://github.com/jckantor/ND-Pyomo-Cookbook/blob/master/notebooks/03.00-Assignment-Problems.ipynb

-------------------------------------------------------------------------

In [ ]:
# import modules

%matplotlib inline
from pylab import *

import shutil
import sys
import os.path

if not shutil.which("pyomo"):
    !pip install -q pyomo
    assert(shutil.which("pyomo"))

if not (shutil.which("cbc") or os.path.isfile("cbc")):
    if "google.colab" in sys.modules:
        !apt-get install -y -qq coinor-cbc
    else:
        try:
            !conda install -c conda-forge coincbc 
        except:
            pass

assert(shutil.which("cbc") or os.path.isfile("cbc"))

from pyomo.environ import *
# ensure you have cbc installed
!apt-get install -y -qq coinor-cbc

# Example
***(6. Assigning Engineers)*** A small engineering firm has 4 senior
designers available to work on the firm’s 4 current projects over
the next 2 weeks. The firm’s manager has developed the following table of quality scores, which show each designer’s
design quality on each type of project, on a scale of 100. Also
shown is an estimate of the time (in hours) required for each
project:

Person | Project1 | Project2 | Project3 | Project4 | 
--- | --- | --- | --- | --- | 
Employee1 | 90 | 80 | 25 | 50 | 
Employee2 | 60 | 70 | 50 | 65 | 
Employee3 | 70 | 40 | 80 | 85 | 
Employee4 | 65 | 55 | 60 | 75 | 
Time | 70|50|85|35

1. Assume that one designer is assigned to each project. What
assignment of designers to projects maximizes the sum of the
quality scores assigned?
2. Suppose that each designer has 80 hours available over the
next two weeks. Assuming that more than one designer can
work on a project, what assignment schedule maximizes the
sum of quality scores assigned?



Let's go after part 1 first...

**Objective Function:** $\max(\text{Quality}) = \\
\quad 90_{E_1P_1} + 80_{E_1P_2} + 25_{E_1P_3} + 50_{E_1P_4} + \\
\quad 60_{E_2P_1} + 70_{E_2P_2} + 50_{E_2P_3} + 65_{E_2P_4} + \\
\quad 70_{E_3P_1} + 40_{E_3P_2} + 80_{E_3P_3} + 85_{E_3P_4} + \\
\quad 65_{E_4P_1} + 55_{E_4P_2} + 60_{E_4P_3} + 75_{E_4P_4} $

Subject to constraints:

*(that each employee ($E$) can only do one project ($P$)! notice that these are the rows of the table...)*

$E_1P_1 + E_1P_2 + E_1P_3 + E_1P_4 \leq 1$

$E_2P_1 + E_2P_2 + E_2P_3 + E_2P_4 \leq 1$

$E_3P_1 + E_3P_2 + E_3P_3 + E_3P_4 \leq 1$

$E_4P_1 + E_4P_2 + E_4P_3 + E_4P_4 \leq 1$

*(that each project($P$) needs one employee ($E$)! notice that these are the columns of the table...)*

$E_1P_1 + E_2P_1 + E_3P_1 + E_4P_1 \geq 1$

$E_1P_2 + E_2P_2 + E_3P_2 + E_4P_2 \geq 1$

$E_1P_3 + E_2P_3 + E_3P_3 + E_4P_3 \geq 1$

$E_1P_4 + E_2P_4 + E_3P_4 + E_4P_4 \geq 1$

# Solved Like Powell

In [ ]:
# declare the model
model = ConcreteModel()

# write your variables - 

# for Part 2, you need real non-neg reals...

# EMPLOYEE 1
model.E1P1 = Var(domain=NonNegativeReals)
model.E1P2 = Var(domain=NonNegativeReals)
model.E1P3 = Var(domain=NonNegativeReals)
model.E1P4 = Var(domain=NonNegativeReals)
# EMPLOYEE 2
model.E2P1 = Var(domain=NonNegativeReals)
model.E2P2 = Var(domain=NonNegativeReals)
model.E2P3 = Var(domain=NonNegativeReals)
model.E2P4 = Var(domain=NonNegativeReals)
# EMPLOYEE 3
model.E3P1 = Var(domain=NonNegativeReals)
model.E3P2 = Var(domain=NonNegativeReals)
model.E3P3 = Var(domain=NonNegativeReals)
model.E3P4 = Var(domain=NonNegativeReals)
# EMPLOYEE 4
model.E4P1 = Var(domain=NonNegativeReals)
model.E4P2 = Var(domain=NonNegativeReals)
model.E4P3 = Var(domain=NonNegativeReals)
model.E4P4 = Var(domain=NonNegativeReals)

In [ ]:
# write the objective function
model.OBJ = Objective(expr = 90*model.E1P1 + 80*model.E1P2 + 25*model.E1P3 + 50*model.E1P4 +
                            60*model.E2P1 + 70*model.E2P2 + 50*model.E2P3 + 65*model.E2P4 + 
                            70*model.E3P1 + 40*model.E3P2 + 80*model.E3P3 + 85*model.E3P4 +
                            65*model.E4P1 + 55*model.E4P2 + 60*model.E4P3 + 75*model.E4P4,
                      sense=maximize) # remember, you are maximizing quality!

In [ ]:
# write the constraints

# 'each employee should be assigned to one project'
model.OneProj_E1 = Constraint(expr = model.E1P1 + model.E1P2 + model.E1P3 + model.E1P4 == 1)
model.OneProj_E2 = Constraint(expr = model.E2P1 + model.E2P2 + model.E2P3 + model.E2P4 == 1)
model.OneProj_E3 = Constraint(expr = model.E3P1 + model.E3P2 + model.E3P3 + model.E3P4 == 1)
model.OneProj_E4 = Constraint(expr = model.E4P1 + model.E4P2 + model.E4P3 + model.E4P4 == 1)

# 'each project be assigned one person' (note equality constraint, ==)
model.Do_Proj1 = Constraint(expr = model.E1P1 + model.E2P1 + model.E3P1 + model.E4P1 == 1)
model.Do_Proj2 = Constraint(expr = model.E1P2 + model.E2P2 + model.E3P2 + model.E4P2 == 1)
model.Do_Proj3 = Constraint(expr = model.E1P3 + model.E2P3 + model.E3P3 + model.E4P3 == 1)
model.Do_Proj4 = Constraint(expr = model.E1P4 + model.E2P4 + model.E3P4 + model.E4P4 == 1)

# for Part 2... note that time was ignored before...
# time must be less than 80 but you can work on multiple projects
# model.TimeConstraintE1 = Constraint(expr = 70*model.E1P1 + 50*model.E1P2 + 85*model.E1P3 + 35*model.E1P4 <= 80) # E1 can only work 80 hrs
# model.TimeConstraintE2 = Constraint(expr = 70*model.E2P1 + 50*model.E2P2 + 85*model.E2P3 + 35*model.E2P4 <= 80) # E2 can only work 80 hrs
# model.TimeConstraintE3 = Constraint(expr = 70*model.E3P1 + 50*model.E3P2 + 85*model.E3P3 + 35*model.E3P4 <= 80) # ...
# model.TimeConstraintE4 = Constraint(expr = 70*model.E4P1 + 50*model.E4P2 + 85*model.E4P3 + 35*model.E4P4 <= 80)

Pretty print to inspect.

In [ ]:
model.pprint()

16 Var Declarations
    E1P1 : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 :  None :  None : False :  True : NonNegativeReals
    E1P2 : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 :  None :  None : False :  True : NonNegativeReals
    E1P3 : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 :  None :  None : False :  True : NonNegativeReals
    E1P4 : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 :  None :  None : False :  True : NonNegativeReals
    E2P1 : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 :  None :  None : False :  True : NonNegativeReals
    E2P2 : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 :  None :  None : False :  True : NonNegativeRe

Now solve it!

In [ ]:
# Solve the model
SolverFactory('cbc', executable='/usr/bin/cbc').solve(model).write()

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: unknown
  Lower bound: 314.0
  Upper bound: 314.0
  Number of objectives: 1
  Number of constraints: 13
  Number of variables: 17
  Number of nonzeros: 16
  Sense: maximize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  User time: -1.0
  System time: 0.0
  Wallclock time: 0.0
  Termination condition: optimal
  Termination message: Model was solved to optimality (subject to tolerances), and an optimal solution is available.
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: None
      Number of created 

Show the results.

In [ ]:
# show the results (quality is 315 in original case, only 314 in part 2 due to time costraints... more constraints = less optimal!)
print('Total QUALITY:',model.OBJ())

Total QUALITY: 314.0


In [ ]:
# nice way to show assignments?
print("E1:",model.E1P1(), model.E1P2 (), model.E1P3(), model.E1P4())
print("E2:",model.E2P1(), model.E2P2 (), model.E2P3(), model.E2P4())
print("E3:",model.E3P1(), model.E3P2 (), model.E3P3(), model.E3P4())
print("E4:",model.E4P1(), model.E4P2 (), model.E4P3(), model.E4P4())

# you could turn this into a pandas dataframe if you wanted to (left to students!)

E1: 1.0 0.0 0.0 0.0
E2: 0.0 1.0 0.0 0.0
E3: 0.0 0.0 0.9 0.1
E4: 0.0 0.0 0.1 0.9


In [ ]:
# try it on your own
E1 = [model.E1P1(), model.E1P2 (), model.E1P3(), model.E1P4()]
E2 = [model.E2P1(), model.E2P2 (), model.E2P3(), model.E2P4()]
E3 = [model.E3P1(), model.E3P2 (), model.E3P3(), model.E3P4()]
E4 = [model.E4P1(), model.E4P2 (), model.E4P3(), model.E4P4()]

In [ ]:
# almost there! just need to rename rownames and column names... could color code?!
import pandas as pd
df = pd.DataFrame([E1, E2, E3, E4])
df.rename({0:'P1', 1:'P2', 2:'P3', 3:'P4'}, axis=1, inplace=True) # axis=1 means columns
df.rename({0:'E1', 1:'E2', 2:'E3', 3:'E4'}, axis=0, inplace=True) # axis=0 means rows
df

,P1,P2,P3,P4
E1,1.0,0.0,0.0,0.0
E2,0.0,1.0,0.0,0.0
E3,0.0,0.0,0.9,0.1
E4,0.0,0.0,0.1,0.9


# On Your Own
You may try to run a sensitivity analysis on what you see. Yes, it takes a long time to code up problems like this, but it is easy to tinker with the for loop rather than limiting yourself to the 'Sensitivity Report'.

You may also attempt Part 2 by switching to NonNegativeReals and relaxing the inequality constraint (let people work on at least 4 projects!) Just uncomment the cells and re-run. Note how we use `NonNegativeReals` instead of `NonNegativeIntegers`.


# Solved According To Pyomo Cookbook

In [ ]:
Quality = {
    ('E1','P1'): 90,
    ('E1','P2'): 80,
    ('E1','P3'): 25,
    ('E1','P4'): 50,
    ('E2','P1'): 60,
    ('E2','P2'): 70,
    ('E2','P3'): 50,
    ('E2','P4'): 65,
    ('E3','P1'): 70,
    ('E3','P2'): 40,
    ('E3','P3'): 80,
    ('E3','P4'): 85,
    ('E4','P1'): 65,
    ('E4','P2'): 55,
    ('E4','P3'): 60,
    ('E4','P4'): 75
}

assignments = list(Quality.keys())

machines = ('E1','E2','E3','E4')
jobs = ('P1','P2','P3','P4')

In [ ]:
def create_bounds(model, i, j):
   return (0,1)

# declare the model
model = ConcreteModel()

# Create variables
model.x = Var(machines, jobs, domain = NonNegativeReals, bounds=create_bounds)

# Constraints
model.machine_constraints = ConstraintList()

# At most one job per machine - for loop is much more compact notation
for machine in machines:
  assign_expr = 0
  for job in jobs:
    assign_expr += model.x[machine,job]
  model.machine_constraints.add(assign_expr <= 1)


# Exactly one machine per job - for loop (again) is much more compact!
model.job_constraints = ConstraintList()
for job in jobs:
  assign_expr = 0
  for machine in machines:
    assign_expr += model.x[machine,job]
  model.job_constraints.add(assign_expr == 1)


# Objective - nice compact way (again!)
obj_expr = 0.0
for quality in Quality:
  print(quality,Quality[quality])
  obj_expr += Quality[quality]*model.x[quality]

model.OBJ = Objective(
    expr = obj_expr, 
    sense = maximize)

model.pprint()

('E1', 'P1') 90
('E1', 'P2') 80
('E1', 'P3') 25
('E1', 'P4') 50
('E2', 'P1') 60
('E2', 'P2') 70
('E2', 'P3') 50
('E2', 'P4') 65
('E3', 'P1') 70
('E3', 'P2') 40
('E3', 'P3') 80
('E3', 'P4') 85
('E4', 'P1') 65
('E4', 'P2') 55
('E4', 'P3') 60
('E4', 'P4') 75
5 Set Declarations
    job_constraints_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    4 : {1, 2, 3, 4}
    machine_constraints_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    4 : {1, 2, 3, 4}
    x_index : Size=1, Index=None, Ordered=True
        Key  : Dimen : Domain              : Size : Members
        None :     2 : x_index_0*x_index_1 :   16 : {('E1', 'P1'), ('E1', 'P2'), ('E1', 'P3'), ('E1', 'P4'), ('E2', 'P1'), ('E2', 'P2'), ('E2', 'P3'), ('E2', 'P4'), ('E3', 'P1'), ('E3', 'P2'), ('E3', 'P3'), ('E3', 'P4'), ('E4', 'P1'), ('E4', 'P2'), ('E4', 'P3'), ('E4', 'P4')}
    x_i

In [ ]:
# Solve the model
SolverFactory('cbc', executable='/usr/bin/cbc').solve(model).write()

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: unknown
  Lower bound: 315.0
  Upper bound: 315.0
  Number of objectives: 1
  Number of constraints: 9
  Number of variables: 17
  Number of nonzeros: 16
  Sense: maximize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  User time: -1.0
  System time: 0.0
  Wallclock time: 0.0
  Termination condition: optimal
  Termination message: Model was solved to optimality (subject to tolerances), and an optimal solution is available.
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: None
      Number of created s

In [ ]:
# show the results
print('Total Quality:',model.OBJ())

print("List of assigments")
for assignment in assignments:
  if 0 < model.x[assignment]():
    print(assignment)

Total Quality: 315.0
List of assigments
('E1', 'P1')
('E2', 'P2')
('E3', 'P3')
('E4', 'P4')


# On Your Own
Try running the Pyomo cookbook example for Part 2...